In [1]:
from ai import ai_abs_dls, ai14s
from marubatsu import Marubatsu

mb = Marubatsu()
eval_params = {"minimax": True}

In [2]:
%%timeit

ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=9)

68.7 ms ± 1.76 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [3]:
from ai import show_progress, ai2, ai11s

show_progress(ai=[ai2, ai11s], winner=Marubatsu.CIRCLE)

TypeError: cannot unpack non-iterable NoneType object

In [4]:
def show_progress(ai, winner, params=[{}, {}]):
    mb = Marubatsu()
    while True:
        if mb.play(ai=ai, verbose=False, params=params) == winner:
            records = mb.records
            mb.restart()
            for x, y in records[1:]:
                mb.move(x, y)
                print(mb)
            break

In [5]:
show_progress(ai=[ai2, ai11s], winner=Marubatsu.CIRCLE)

Turn x
..O
...
...

Turn o
..o
.X.
...

Turn x
..o
.x.
O..

Turn o
..o
.x.
o.X

Turn x
O.o
.x.
o.x

Turn o
o.o
.xX
o.x

winner o
o.o
Oxx
o.x



In [6]:
mb = Marubatsu()
print(mb.records)

[None]


In [7]:
def restart(self):
    self.initialize_board()
    self.turn = Marubatsu.CIRCLE     
    self.move_count = 0
    self.status = Marubatsu.PLAYING
    self.last_move = -1, -1          
    self.last_turn = None
    self.records = [self.last_move]
    
Marubatsu.restart = restart

In [8]:
mb = Marubatsu()
print(mb.records)

[(-1, -1)]


In [9]:
def remove_mark(self, x, y):
    if 0 <= x < self.BOARD_SIZE and 0 <= y < self.BOARD_SIZE:
        if self.board[x][y] != Marubatsu.EMPTY:
            self.board[x][y] = Marubatsu.EMPTY
            return True
        else:
            print("(", x, ",", y, ") のマスにはマークがありません")
            return False
    else:
        print("(", x, ",", y, ") はゲーム盤の範囲外の座標です")
        return False   
    
Marubatsu.remove_mark = remove_mark

In [10]:
mb = Marubatsu()
mb.move(1, 1)
print(mb)
mb.remove_mark(1, 1)
print(mb)

Turn x
...
.O.
...

Turn x
...
...
...



In [11]:
a = [1, 2, 3]
b = a.pop()
print(a)
print(b)

[1, 2]
3


In [12]:
a = [1, 2, 3]
b = a.pop(1)
print(a)
print(b)

[1, 3]
2


In [13]:
a = {
    "x": 1,
    "y": 2,
    "z": 3,
}
b = a.pop("z")
print(a)
print(b)

{'x': 1, 'y': 2}
3


In [14]:
def unmove(self):
    if self.move_count > 0:
        self.move_count -= 1
        self.turn, self.last_turn = self.last_turn, self.turn
        if self.move_count == 0:
            self.last_move = (-1, -1)
        self.status = Marubatsu.PLAYING
        x, y = self.records.pop()
        self.remove_mark(x, y)
        self.last_move = self.records[-1]
            
Marubatsu.unmove = unmove

In [15]:
mb = Marubatsu()
mb.move(0, 0)
mb.move(1, 0)
mb.move(2, 0)
mb.move(0, 1)
mb.move(1, 1)
for _ in range(5):
    mb.unmove()
    print(mb)

Turn o
oxo
X..
...

Turn x
oxO
...
...

Turn o
oX.
...
...

Turn x
O..
...
...

Turn o
...
...
...



In [16]:
from ai import ai_by_mmscore, dprint
from time import perf_counter

@ai_by_mmscore
def ai_abs_dls(mb, debug=False, timelimit_pc=None, maxdepth=1, eval_func=None,
               eval_params={}, use_tt=False, tt=None):           
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound, _ = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mborig.move(x, y)
                abscore = ab_search(mborig, depth + 1, tt, alpha, beta)
                mborig.unmove()
                if abscore > score:
                    bestmove = (x, y)
                score = max(score, abscore)
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mborig.move(x, y)
                abscore = ab_search(mb, depth + 1, tt, alpha, beta)
                mborig.unmove()
                if abscore < score:
                    bestmove = (x, y)
                score = min(score, abscore)
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
        from util import calc_same_boardtexts

        if use_tt:
            boardtxtlist = calc_same_boardtexts(mborig, bestmove)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt, move in boardtxtlist.items():
                tt[boardtxt] = (lower_bound, upper_bound, move)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [17]:
%%timeit

ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, analyze=True, use_tt=True, maxdepth=9)

35.3 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
from util import Check_solved

params = {
    "eval_func": ai14s,
    "eval_params": eval_params,
    "use_tt": True,
    "maxdepth": 9
}

Check_solved.is_strongly_solved(ai_abs_dls, params=params)

  0%|          | 0/431 [00:00<?, ?it/s]

100%|██████████| 431/431 [00:01<00:00, 379.45it/s]

431/431 100.00%


(True, [])

In [19]:
from ai import ai_match, ai2s

ai_match(ai=[ai2s, ai14s])

ai2s VS ai14s


100%|██████████| 10000/10000 [01:08<00:00, 145.00it/s]

count     win    lose    draw
o           0    8904    1096
x           0    9895     105
total       0   18799    1201

ratio     win    lose    draw
o        0.0%   89.0%   11.0%
x        0.0%   99.0%    1.1%
total    0.0%   94.0%    6.0%



[('count',
  [{'win': 0, 'lose': 8904, 'draw': 1096},
   {'win': 0, 'lose': 9895, 'draw': 105},
   {'win': 0, 'lose': 18799, 'draw': 1201}],
  '7d'),
 ('ratio',
  [{'win': 0.0, 'lose': 0.8904, 'draw': 0.1096},
   {'win': 0.0, 'lose': 0.9895, 'draw': 0.0105},
   {'win': 0.0, 'lose': 0.93995, 'draw': 0.06005}],
  '7.1%')]

In [20]:
from functools import wraps
from random import choice

def ai_by_score(eval_func):
    @wraps(eval_func)
    def wrapper(mb_orig, debug=False, *args, rand=True, 
                analyze=False, calc_score=False, minimax=False, **kwargs):
        if calc_score:
            score = eval_func(mb_orig, debug, *args, **kwargs)
            if minimax and mb_orig.turn == Marubatsu.CIRCLE:
                score *= -1
            return score

        dprint(debug, "Start ai_by_score")
        dprint(debug, mb_orig)
        legal_moves = mb_orig.calc_legal_moves()
        dprint(debug, "legal_moves", legal_moves)
        best_score = float("-inf")
        best_moves = []
        if analyze:
            score_by_move = {}
        for move in legal_moves:
            dprint(debug, "=" * 20)
            dprint(debug, "move", move)
            x, y = move
            mb_orig.move(x, y)
            dprint(debug, mb_orig)
            score = eval_func(mb_orig, debug, *args, **kwargs)
            mb_orig.unmove()
            dprint(debug, "score", score, "best score", best_score)
            if analyze:
                score_by_move[move] = score
            
            if best_score < score:
                best_score = score
                best_moves = [move]
                dprint(debug, "UPDATE")
                dprint(debug, "  best score", best_score)
                dprint(debug, "  best moves", best_moves)
            elif best_score == score:
                best_moves.append(move)
                dprint(debug, "APPEND")
                dprint(debug, "  best moves", best_moves)

        dprint(debug, "=" * 20)
        dprint(debug, "Finished")
        dprint(debug, "best score", best_score)
        dprint(debug, "best moves", best_moves)
        if analyze:
            return {
                "candidate": best_moves,
                "score_by_move": score_by_move,
            }
        elif rand:   
            return choice(best_moves)
        else:
            return best_moves[0]
        
    return wrapper

In [21]:
from marubatsu import Markpat

@ai_by_score
def ai2s(mb, debug=False):
    return 0

@ai_by_score
def ai14s(mb, debug=False, score_victory=300, score_sure_victory=200, \
          score_defeat=-100, score_special=100, score_201=2, \
          score_102=0.5, score_012=-1):
    # 評価値の合計を計算する変数を 0 で初期化する
    score = 0        

    # 自分が勝利している場合
    if mb.status == mb.last_turn:
        return score_victory

    markpats = mb.count_markpats()
    if debug:
        pprint(markpats)
    # 相手が勝利できる場合は評価値を加算する
    if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
        score = score_defeat * markpats[Markpat(last_turn=0, turn=2, empty=1)]
    # 次の自分の手番で自分が必ず勝利できる場合
    elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
        return score_sure_victory
    
    # 斜め方向に 〇×〇 が並び、いずれかの辺の 1 つのマスのみに × が配置されている場合
    if mb.board[1][1] == Marubatsu.CROSS and \
        (mb.board[0][0] == mb.board[2][2] == Marubatsu.CIRCLE or \
        mb.board[2][0] == mb.board[0][2] == Marubatsu.CIRCLE) and \
        (mb.board[1][0] == Marubatsu.CROSS or \
        mb.board[0][1] == Marubatsu.CROSS or \
        mb.board[2][1] == Marubatsu.CROSS or \
        mb.board[1][2] == Marubatsu.CROSS) and \
        mb.move_count == 4:
        return score_special    

    # 次の自分の手番で自分が勝利できる場合は評価値に score_201 を加算する
    if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
        score += score_201
    # 「自 1 敵 0 空 2」1 つあたり score_102 だけ、評価値を加算する
    score += markpats[Markpat(last_turn=1, turn=0, empty=2)] * score_102
    # 「自 0 敵 1 空 2」1 つあたり score_201 だけ、評価値を減算する
    score += markpats[Markpat(last_turn=0, turn=1, empty=2)] * score_012
    
    # 計算した評価値を返す
    return score

In [22]:
ai_match(ai=[ai2s, ai14s])

ai2s VS ai14s


100%|██████████| 10000/10000 [00:37<00:00, 266.91it/s]

count     win    lose    draw
o           0    8783    1217
x           0    9905      95
total       0   18688    1312

ratio     win    lose    draw
o        0.0%   87.8%   12.2%
x        0.0%   99.1%    0.9%
total    0.0%   93.4%    6.6%



[('count',
  [{'win': 0, 'lose': 8783, 'draw': 1217},
   {'win': 0, 'lose': 9905, 'draw': 95},
   {'win': 0, 'lose': 18688, 'draw': 1312}],
  '7d'),
 ('ratio',
  [{'win': 0.0, 'lose': 0.8783, 'draw': 0.1217},
   {'win': 0.0, 'lose': 0.9905, 'draw': 0.0095},
   {'win': 0.0, 'lose': 0.9344, 'draw': 0.0656}],
  '7.1%')]